In [1]:
#installing kaggle library
! pip install kaggle


In [2]:
# configuring path of kaggle.json file
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [3]:
#api command
!kaggle datasets download -d bwandowando/3-million-instagram-google-store-reviews


Dataset URL: https://www.kaggle.com/datasets/bwandowando/3-million-instagram-google-store-reviews
License(s): CC0-1.0
3-million-instagram-google-store-reviews.zip: Skipping, found more recently modified local copy (use --force to force download)


In [4]:
#extracting zip file
from zipfile import ZipFile
dataset = '/content/3-million-instagram-google-store-reviews.zip'
with ZipFile(dataset,'r') as zip:
  zip.extractall()
  print('extracted')

extracted


In [6]:
!pip install emoji


In [7]:
#required libraries
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [8]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [9]:
sw = set(stopwords.words('english')) - {'not', 'no', 'nor'}
print(sw)

{'is', 'having', 'whom', 'will', 'these', 'both', 'had', 'isn', 'into', 'how', 'on', 'herself', 'their', 's', 'here', 'as', 'his', 'theirs', 'have', "we're", "i'm", 'yourself', 'who', 'too', "hasn't", "needn't", 'just', 'itself', 'am', 'own', 'to', 'you', 'wasn', 'your', 'hasn', 't', 'those', "couldn't", 'did', 'o', 'before', "hadn't", "she'd", 'yours', 'any', "i'd", 'until', 'for', 're', 'from', 'do', 'at', 'through', 'only', 'by', 'again', 'over', "it'd", 'yourselves', "mustn't", "won't", "i'll", 'didn', 'ours', 'once', 'where', 'our', 'other', 'd', "she'll", "he's", "weren't", 'off', 'couldn', 'down', 'were', 'i', 'haven', 'y', 'are', "we'll", 'about', 'shan', 'during', "don't", "you're", "mightn't", 'hers', 'aren', "i've", 'me', 'some', 'my', "we'd", 'him', 'doesn', 'them', 'under', "it'll", 'needn', 'up', "we've", 'now', 'should', 'was', "you'd", 'that', "you've", "that'll", 'which', 'myself', 'll', "they've", "wasn't", 'same', 'if', 'ain', 'such', 'few', 'and', 'after', 'being', 

In [10]:
df=pd.read_csv('/content/INSTAGRAM_REVIEWS.csv',encoding= 'ISO-8859-1')

In [11]:
df.shape

(3079939, 9)

In [12]:
df.head()

,Unnamed: 0,review_id,pseudo_author_id,author_name,review_text,review_rating,review_likes,author_app_version,review_timestamp
0,0,ff25650a-a9e1-4c02-a8e8-0c85b9ee1dc9,127325963209728431078,B ****or,Used this app for years. Was a great way to es...,1,0,216.1.0.21.137,2022-01-05 23:37:15
1,1,7e1840fe-4307-4c97-974c-437ec6c64e60,148177168797697591284,Ù Ø³*******ÙÙ,khayemalo sag bgad,5,0,155.0.0.37.107,2022-01-05 23:39:29
2,2,3c8ba6fc-3009-4b1c-9a24-9ea635799a9a,162050472706594017332,Li********on,Lovely,5,0,NaN,2022-01-05 23:39:32
3,3,47c54668-dbac-4f8f-a98a-3ede00a103ea,264067006274581079256,Fa******Zx,best,5,0,NaN,2022-01-05 23:39:50
4,4,464a6a51-9a9f-44b0-80b5-eeea24adcd57,224172402296862948810,fr***************67,Super,5,0,216.0.0.20.137,2022-01-05 23:40:08


In [13]:
df.isnull().sum()

,0
Unnamed: 0,0
review_id,0
pseudo_author_id,0
author_name,0
review_text,128
review_rating,0
review_likes,0
author_app_version,765552
review_timestamp,0


In [14]:
df.dropna(subset=['review_text'], inplace=True)  # Drop missing reviews

df['author_app_version'] = df['author_app_version'].fillna('unknown') # Fill Nan Values

In [15]:
df.isnull().sum()


,0
Unnamed: 0,0
review_id,0
pseudo_author_id,0
author_name,0
review_text,0
review_rating,0
review_likes,0
author_app_version,0
review_timestamp,0


In [16]:
df.shape

(3079811, 9)

In [17]:
df.head()

,Unnamed: 0,review_id,pseudo_author_id,author_name,review_text,review_rating,review_likes,author_app_version,review_timestamp
0,0,ff25650a-a9e1-4c02-a8e8-0c85b9ee1dc9,127325963209728431078,B ****or,Used this app for years. Was a great way to es...,1,0,216.1.0.21.137,2022-01-05 23:37:15
1,1,7e1840fe-4307-4c97-974c-437ec6c64e60,148177168797697591284,Ù Ø³*******ÙÙ,khayemalo sag bgad,5,0,155.0.0.37.107,2022-01-05 23:39:29
2,2,3c8ba6fc-3009-4b1c-9a24-9ea635799a9a,162050472706594017332,Li********on,Lovely,5,0,unknown,2022-01-05 23:39:32
3,3,47c54668-dbac-4f8f-a98a-3ede00a103ea,264067006274581079256,Fa******Zx,best,5,0,unknown,2022-01-05 23:39:50
4,4,464a6a51-9a9f-44b0-80b5-eeea24adcd57,224172402296862948810,fr***************67,Super,5,0,216.0.0.20.137,2022-01-05 23:40:08


In [18]:
df['review_rating'].value_counts()

,count
review_rating,
5,1963214
1,589844
4,246602
3,155200
2,124951


In [19]:
#labelling the ratings
def label_sentiment(rating):
    if rating >= 4:
        return 'positive review'
    elif rating == 3:
        return 'neutral review'
    else:
        return 'negative review'

In [20]:
df['sentiment'] = df['review_rating'].apply(label_sentiment)
print(df['sentiment'].value_counts())

sentiment
positive review    2209816
negative review     714795
neutral review      155200
Name: count, dtype: int64


In [21]:
import emoji

def convert_emojis(text):
    return emoji.demojize(text, delimiters=(" ", " "))


In [22]:
#port_stem=PorterStemmer()
#def stemming(content):
 # stemmed_content=re.sub('[^a-zA-Z]',' ',content)
  #stemmed_content=stemmed_content.lower()
  #stemmed_content=stemmed_content.split()
 # stemmed_content=[port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  #stemmed_content=' '.join(stemmed_content)
 # return stemmed_content

In [23]:
lemmatizer = WordNetLemmatizer()
custom_stopwords = set(stopwords.words('english')) - {'not', 'no', 'nor'}

def lemmatize_review(content):
    content = convert_emojis(content)
    cleaned = re.sub('[^a-zA-Z]', ' ', content)  # Keep only letters
    cleaned = cleaned.lower()
    words = cleaned.split()
    words = [lemmatizer.lemmatize(word) for word in words if word not in custom_stopwords]
    return ' '.join(words)


In [24]:
df['stemmed_content']=df['review_text'].apply(lemmatize_review)

In [25]:
# Clean missing stemmed_content first
df = df[df['stemmed_content'].notna()]
df = df[df['stemmed_content'].str.strip() != '']


In [26]:
df.head()

,Unnamed: 0,review_id,pseudo_author_id,author_name,review_text,review_rating,review_likes,author_app_version,review_timestamp,sentiment,stemmed_content
0,0,ff25650a-a9e1-4c02-a8e8-0c85b9ee1dc9,127325963209728431078,B ****or,Used this app for years. Was a great way to es...,1,0,216.1.0.21.137,2022-01-05 23:37:15,negative review,used app year great way escape overly politica...
1,1,7e1840fe-4307-4c97-974c-437ec6c64e60,148177168797697591284,Ù Ø³*******ÙÙ,khayemalo sag bgad,5,0,155.0.0.37.107,2022-01-05 23:39:29,positive review,khayemalo sag bgad
2,2,3c8ba6fc-3009-4b1c-9a24-9ea635799a9a,162050472706594017332,Li********on,Lovely,5,0,unknown,2022-01-05 23:39:32,positive review,lovely
3,3,47c54668-dbac-4f8f-a98a-3ede00a103ea,264067006274581079256,Fa******Zx,best,5,0,unknown,2022-01-05 23:39:50,positive review,best
4,4,464a6a51-9a9f-44b0-80b5-eeea24adcd57,224172402296862948810,fr***************67,Super,5,0,216.0.0.20.137,2022-01-05 23:40:08,positive review,super


In [27]:
from sklearn.utils import resample

df_pos = df[df['sentiment'] == 'positive review']
df_neg =df[df['sentiment'] == 'negative review']
df_neu = df[df['sentiment'] == 'neutral review']

df_neg_upsampled = resample(df_neg,
                            replace=True,
                            n_samples=len(df_pos),
                            random_state=42)

df_neu_upsampled = resample(df_neu,
                            replace=True,
                            n_samples=len(df_pos),
                            random_state=42)

df_balanced = pd.concat([df_pos, df_neg_upsampled, df_neu_upsampled])

# ✅ Sample here to reduce runtime (e.g., during development)
df_balanced = df_balanced.sample(5000, random_state=42)

print("Sampled sentiment class counts:")
print(df_balanced['sentiment'].value_counts())



Sampled sentiment class counts:
sentiment
neutral review     1678
positive review    1673
negative review    1649
Name: count, dtype: int64


In [28]:
vectorizer = TfidfVectorizer(ngram_range=(1,2), max_features=20000)
X = vectorizer.fit_transform(df_balanced['stemmed_content'])



In [29]:
#import spacy
#nlp = spacy.load('en_core_web_md')

#texts = df_balanced['stemmed_content'].tolist()

# ##Fast batch vectorization
#doc_vectors = []
#for doc in nlp.pipe(texts, batch_size=512, disable=["ner", "parser", "tagger"]):
 #   doc_vectors.append(doc.vector)

#X = np.vstack(doc_vectors)


In [30]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = le.fit_transform(df_balanced['sentiment'])


In [31]:
print(X)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 46689 stored elements and shape (5000, 20000)>
  Coords	Values
  (0, 11132)	0.3352601877548772
  (0, 19699)	0.35310007729770093
  (0, 642)	0.19816631317928432
  (0, 11186)	0.6841165295108124
  (0, 19701)	0.5056042209969511
  (1, 642)	0.03782916210051353
  (1, 19674)	0.12459441526068708
  (1, 8773)	0.04752319376413777
  (1, 1132)	0.11433612859730478
  (1, 19832)	0.09444941965899278
  (1, 139)	0.09982026829026679
  (1, 9115)	0.10179647672408951
  (1, 7342)	0.0841911329956105
  (1, 10418)	0.10289324353453906
  (1, 5757)	0.294458254170708
  (1, 11942)	0.11433612859730478
  (1, 4575)	0.13059512828041367
  (1, 9186)	0.10833541557757818
  (1, 13476)	0.09101885329051415
  (1, 13514)	0.07269883359880001
  (1, 5382)	0.12033684161703137
  (1, 15972)	0.059558416528251536
  (1, 11970)	0.21355224854259597
  (1, 256)	0.15725146620487102
  (1, 13289)	0.0989230493923269
  :	:
  (4995, 12983)	0.12684193160864377
  (4995, 10523)	0.240436304093

In [32]:
print(y)

[1 0 1 ... 0 0 1]


In [33]:


X_train, X_test, y_train, y_test, idx_train, idx_test = train_test_split(
    X, y, df_balanced.index,
    test_size=0.2,
    stratify=y,
    random_state=42
)


In [34]:
print(X.shape,X_train.shape,X_test.shape)

(5000, 20000) (4000, 20000) (1000, 20000)


In [35]:
print(X_train.shape[0], y_train.shape[0])


4000 4000


In [36]:
print(X_train)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 37203 stored elements and shape (4000, 20000)>
  Coords	Values
  (0, 642)	0.07905425193208869
  (0, 12983)	0.09925616345743701
  (0, 19890)	0.14241404870139865
  (0, 13208)	0.15131139880792954
  (0, 19665)	0.17731490415932893
  (0, 19666)	0.22018982568806142
  (0, 18526)	0.15383751394521472
  (0, 11747)	0.30323221571452413
  (0, 15696)	0.3921226031286123
  (0, 8316)	0.251476333750874
  (0, 15703)	0.44627510416682553
  (0, 11772)	0.2729137945152402
  (0, 8319)	0.2729137945152402
  (0, 930)	0.23893622309303866
  (0, 18566)	0.2729137945152402
  (0, 11789)	0.22018982568806142
  (1, 642)	0.29455541477737257
  (1, 19912)	0.626869356471656
  (1, 19913)	0.7212987713438462
  (2, 642)	0.061794342891555734
  (2, 256)	0.12843598011825613
  (2, 12983)	0.07758557255159539
  (2, 19890)	0.11132079986777797
  (2, 13208)	0.1182755921765016
  (2, 18650)	0.10863188990197835
  :	:
  (3993, 17978)	0.2964534778656473
  (3993, 14240)	0.362179461908

In [37]:
print(X_test)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 9486 stored elements and shape (1000, 20000)>
  Coords	Values
  (0, 12732)	1.0
  (1, 12658)	0.3209675344546841
  (1, 18373)	0.3745290676728144
  (1, 15306)	0.3745290676728144
  (1, 8660)	0.39256712698669316
  (1, 18375)	0.39256712698669316
  (1, 15307)	0.39256712698669316
  (1, 8661)	0.39256712698669316
  (2, 12732)	1.0
  (3, 8773)	0.12961394156234815
  (3, 19640)	0.16891069086266836
  (3, 11436)	0.1919402309184065
  (3, 19875)	0.19334434181675772
  (3, 2848)	0.23141570475140025
  (3, 19670)	0.2558818419190038
  (3, 12080)	0.25939397941695547
  (3, 17234)	0.2873722540825107
  (3, 19672)	0.33981666591566956
  (3, 2877)	0.33981666591566956
  (3, 17244)	0.35618290742989606
  (3, 11460)	0.35618290742989606
  (3, 12097)	0.35618290742989606
  (4, 12983)	0.22163442496043806
  (4, 7187)	0.20485405519654937
  (4, 7268)	0.5055320540811747
  :	:
  (995, 15312)	0.4890479746592313
  (995, 10951)	0.49097247526203075
  (995, 10976)	0.72095

In [38]:
print(df.shape)
print(df['review_rating'].shape)

print(df_balanced['sentiment'].shape)


(2908210, 11)
(2908210,)
(5000,)


In [39]:
#print(df_balanced['stemmed_content'].iloc[<index>])


In [40]:
#training the model
model = RandomForestClassifier(n_estimators=100, random_state=42)


In [41]:
model.fit(X_train,y_train)

RandomForestClassifier(random_state=42)

In [42]:
RandomForestClassifier(class_weight='balanced')


RandomForestClassifier(class_weight='balanced')

In [43]:
#accuracy score on training model
X_train_prediction=model.predict(X_train)
training_data_accuracy=accuracy_score(X_train_prediction,y_train)

In [44]:
print('Accuracy score of the training data : ', training_data_accuracy)

Accuracy score of the training data :  0.8815


In [45]:
#accuracy score on test model
X_test_prediction=model.predict(X_test)
test_data_accuracy=accuracy_score(X_test_prediction,y_test)

In [46]:
print('Accuracy score of the test data : ', test_data_accuracy)

Accuracy score of the test data :  0.541


In [47]:
baseline_accuracy = df_balanced['sentiment'].value_counts(normalize=True).max()
print(baseline_accuracy)


0.3356


In [48]:
import pickle

In [49]:
filename='trained_model.sav'
pickle.dump(model,open(filename,'wb'))

In [50]:
#loading saved model
loaded_model=pickle.load(open('trained_model.sav','rb'))


In [51]:
X_new=X_test[2]
print(y_test[2])
prediction = model.predict(X_new.reshape(1, -1))

sentiment = le.inverse_transform(prediction)[0]
print("Predicted Sentiment:", sentiment)



2
Predicted Sentiment: positive review


In [52]:
X_new=X_test[11]
print(y_test[11])
prediction = model.predict(X_new.reshape(1, -1))

sentiment = le.inverse_transform(prediction)[0]
print("Predicted Sentiment:", sentiment)



0
Predicted Sentiment: negative review


In [53]:
X_new=X_test[24]
print(y_test[24])
prediction = model.predict(X_new.reshape(1, -1))

sentiment = le.inverse_transform(prediction)[0]
print("Predicted Sentiment:", sentiment)

1
Predicted Sentiment: positive review


In [54]:
test_idx = idx_test[3]  # This gives actual index from df_balanced
original_review = df_balanced.loc[test_idx, 'review_text']
preprocessed = df_balanced.loc[test_idx, 'stemmed_content']
actual_label = le.inverse_transform([y_test[3]])[0]

print("Original review:", original_review)
print("Preprocessed:", preprocessed)
print("Actual Sentiment:", actual_label)
print("Predicted Sentiment:", sentiment)


Original review: My chat section just doesn't update.. it's been many months but it didn't work after updating instagram..
Preprocessed: chat section update many month work updating instagram
Actual Sentiment: negative review
Predicted Sentiment: positive review


In [61]:
original_index = idx_test[11]
row = df_balanced.loc[original_index]

print("Review Rating     :", row['review_rating'])
print("True Sentiment    :", row['sentiment'])
print("Review Text       :", row['review_text'])

X_new = X_test[11]
proba = model.predict_proba(X_new.reshape(1, -1))

print("Model Confidence  :", proba[0])

prediction = model.predict(X_new.reshape(1, -1))
print("Predicted Sentiment:", le.inverse_transform(prediction)[0])


Review Rating     : 2
True Sentiment    : negative review
Review Text       : Great...up to an extent. It won't let me post multiple photos now. Please fix this
Model Confidence  : [0.6325     0.29856061 0.06893939]
Predicted Sentiment: negative review


In [56]:
proba = model.predict_proba(X_new.reshape(1, -1))

print("Class Probabilities:", proba)
print("Predicted Label:", le.inverse_transform([np.argmax(proba)]))





Class Probabilities: [[0.1624334  0.3114708  0.52609579]]
Predicted Label: ['positive review']


In [57]:
import pickle

# Save the trained model
with open('sentiment_model.pkl', 'wb') as f:
    pickle.dump(model, f)

# Save the vectorizer
with open('vectorizer.pkl', 'wb') as f:
    pickle.dump(vectorizer, f)
